<a href="https://colab.research.google.com/github/Leanse1/56_Secure/blob/main/56_Secure_Category_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## I am doing this as secondary project because I am not properly able to comprehend the Category Mapping part of the assignment.

Reasons for second assignment w.r.t Category Mapping!!!

1.   Conventionally, I seperate test, train dataset as per given dataset classes.
2.   Downloading entire ImageNet dataset is computationally expensive.




## **Plan:** Use ResNet50 independently to predict dataset and then use mapping to predict output w.r.t custom classes. (i.e) 'Cars', 'Bikes', 'Others'

## Importing Dependencies and loading ResNet50 independently as model



In [1]:
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

# Load pre-trained ResNet50 model
model = ResNet50(weights='imagenet')


102967424/102967424 [==============================] - 1s 0us/step


## Building Custom Model w.r.t custom datatset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/56_Secure

/content/gdrive/MyDrive/56_Secure


In [4]:
train_dataset = '/content/gdrive/MyDrive/56_Secure/train'
test_dataset = '/content/gdrive/MyDrive/56_Secure/test'

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dataset,
    target_size=(224, 224),
    batch_size=batch_size
)

test_generator = test_datagen.flow_from_directory(
    test_dataset,
    target_size=(224, 224),
    batch_size=batch_size
)


Found 120 images belonging to 3 classes.
Found 25 images belonging to 3 classes.


In [18]:
BATCH_SIZE = 16
IMAGE_SIZE = 256
CHANNELS=3
EPOCHS=10

In [20]:
train_dataset = '/content/gdrive/MyDrive/56_Secure/train'
test_dataset = '/content/gdrive/MyDrive/56_Secure/test'


# Create a dataset from the directory
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dataset,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset,
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 120 files belonging to 3 classes.
Found 25 files belonging to 3 classes.


In [21]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

# Assuming you have defined IMAGE_SIZE and CHANNELS elsewhere in your code
input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3  # Number of classes

# Load pre-trained ResNet50 model with weights trained on ImageNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the convolutional layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a new model by adding custom layers on top of the base model
ResNet50_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax')  # Adjust to the number of classes
])

ResNet50_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
history = ResNet50_model.fit(
    train_ds,
    batch_size=BATCH_SIZE,
    validation_data=test_ds,
    verbose=1,
    epochs=10,
)

Epoch 1/10
8/8 [==============================] - 12s 708ms/step - loss: 1.0355 - accuracy: 0.5917 - val_loss: 0.5171 - val_accuracy: 0.8000
Epoch 2/10
8/8 [==============================] - 3s 144ms/step - loss: 0.4642 - accuracy: 0.8250 - val_loss: 0.6171 - val_accuracy: 0.7600
Epoch 3/10
8/8 [==============================] - 4s 211ms/step - loss: 0.3070 - accuracy: 0.8583 - val_loss: 0.4395 - val_accuracy: 0.8400
Epoch 4/10
8/8 [==============================] - 4s 148ms/step - loss: 0.1650 - accuracy: 0.9583 - val_loss: 0.5070 - val_accuracy: 0.7600
Epoch 5/10
8/8 [==============================] - 3s 146ms/step - loss: 0.1067 - accuracy: 0.9750 - val_loss: 0.4687 - val_accuracy: 0.8400
Epoch 6/10
8/8 [==============================] - 4s 247ms/step - loss: 0.0769 - accuracy: 0.9917 - val_loss: 0.4598 - val_accuracy: 0.8800
Epoch 7/10
8/8 [==============================] - 4s 147ms/step - loss: 0.0585 - accuracy: 1.0000 - val_loss: 0.4630 - val_accuracy: 0.8800
Epoch 8/10
8/8 [===

In [24]:
# Evaluate on the test set
eval_result = ResNet50_model.evaluate(test_ds)

print("Test Loss:", eval_result[0])
print("Test Accuracy:", eval_result[1])


2/2 [==============================] - 1s 76ms/step - loss: 0.4682 - accuracy: 0.8400
Test Loss: 0.4682376980781555
Test Accuracy: 0.8399999737739563


## Ok !! After using Transfer Learning on our custom Dataset we get loss and accuracy in good range which are some routine things and there is nothing special to explain!!

## Now as I said earlier let me write some codes so that our dataset can get directly predicted on ImageNet dataset!!!

I am using bike test image for predictions

In [38]:
img_path = '/content/gdrive/MyDrive/56_Secure/test/bike/pfva_dataset_file_G_18082023_131243_12.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

predictions = model.predict(img_array)
decoded_predictions = decode_predictions(predictions, top=3)[0]

35363/35363 [==============================] - 0s 0us/step


In [39]:
decoded_predictions

[('n03791053', 'motor_scooter', 0.22540264),
 ('n03444034', 'go-kart', 0.18039659),
 ('n04517823', 'vacuum', 0.14566544)]

## Ok!!  eventhough prediction accuracy is not good still it is able to predict the requisite label

## **Here it comes the time consuming part. I am labelling every class of Imagenet dataset as either car, bike, others**

In [50]:
custom_mapping = {
    0: 'car',
    1: 'bike',
    2: 'others'
}

imagenet_mapping = {
    'sports car': 'car',
    'convertible': 'car',
    'car wheel': 'car',
    'streetcar': 'car',
    'trolleybus': 'car',
    'minibus': 'car',
    'school bus': 'car',
    'passenger car': 'car',
    'electric car': 'car',
    'garbage truck': 'car',
    'pickup truck': 'car',
    'tow truck': 'car',
    'trailer truck': 'car',
    'moving van': 'car',
    'police van': 'car',
    'golfcart': 'car',
    'bicycle': 'bike',
    'mountain bike': 'bike',
    'motorbike': 'bike',
    'dirt bike': 'bike',
    'scooter': 'bike',
     'moped': 'bike',
    'motor_scooter': 'bike',
    'kit_fox': 'others',
    'English_setter': 'others',
    'Siberian_husky': 'others',
    'Australian_terrier': 'others',
    'English_springer': 'others',
    'grey_whale': 'others',
    'lesser_panda': 'others',
    'Egyptian_cat': 'others',
    'ibex': 'others',
    'Persian_cat': 'others',
    'cougar': 'others',
    'gazelle': 'others',
    'porcupine': 'others',
    'sea_lion': 'others',
    'malamute': 'others',
    'badger': 'others',
    'Great_Dane': 'others',
    'Walker_hound': 'others',
    'Welsh_springer_spaniel': 'others',
    'whippet': 'others',
    'Scottish_deerhound': 'others',
    'killer_whale': 'others',
    'mink': 'others',
    'African_elephant': 'others',
    'Weimaraner': 'others',
    'soft-coated_wheaten_terrier': 'others',
    'Dandie_Dinmont': 'others',
    'red_wolf': 'others',
    'Old_English_sheepdog': 'others',
    'jaguar': 'others',
    'otterhound': 'others',
    'bloodhound': 'others',
    'Airedale': 'others',
    'hyena': 'others',
    'meerkat': 'others',
    'giant_schnauzer': 'others',
    'titi': 'others',
    'three-toed_sloth': 'others',
    'sorrel': 'others',
    'black-footed_ferret': 'others',
    'dalmatian': 'others',
    'black-and-tan_coonhound': 'others',
    'papillon': 'others',
    'skunk': 'others',
    'Staffordshire_bullterrier': 'others',
    'Mexican_hairless': 'others',
    'Bouvier_des_Flandres': 'others',
    'weasel': 'others',
    'miniature_poodle': 'others',
    'Cardigan': 'others',
    'malinois': 'others',
    'bighorn': 'others',
    'fox_squirrel': 'others',
    'colobus': 'others',
    'tiger_cat': 'others',
    'Lhasa': 'others',
    'impala': 'others',
    'coyote': 'others',
    'Yorkshire_terrier': 'others',
    'Newfoundland': 'others',
    'brown_bear': 'others',
    'red_fox': 'others',
    'Norwegian_elkhound': 'others',
    'Rottweiler': 'others',
    'hartebeest': 'others',
    'Saluki': 'others',
    'grey_fox': 'others',
    'schipperke': 'others',
    'Pekinese': 'others',
    'Brabancon_griffon': 'others',
    'West_Highland_white_terrier': 'others',
    'Sealyham_terrier': 'others',
    'guenon': 'others',
    'mongoose': 'others',
    'indri': 'others',
    'tiger': 'others',
    'Irish_wolfhound': 'others',
    'wild_boar': 'others',
    'EntleBucher': 'others',
    'zebra': 'others',
    'ram': 'others',
    'French_bulldog': 'others',
    'orangutan': 'others',
    'basenji': 'others',
    'leopard': 'others',
    'Bernese_mountain_dog': 'others',
    'Maltese_dog': 'others',
    'Norfolk_terrier': 'others',
    'toy_terrier': 'others',
    'vizsla': 'others',
    'cairn': 'others',
    'squirrel_monkey': 'others',
    'groenendael': 'others',
    'clumber': 'others',
    'Siamese_cat': 'others',
    'chimpanzee': 'others',
    'komondor': 'others',
    'Afghan_hound': 'others',
    'Japanese_spaniel': 'others',
    'proboscis_monkey': 'others',
    'guinea_pig': 'others',
    'white_wolf': 'others',
    'ice_bear': 'others',
    'gorilla': 'others',
    'borzoi': 'others',
    'toy_poodle': 'others',
    'Kerry_blue_terrier': 'others',
    'ox': 'others',
    'Scotch_terrier': 'others',
    'Tibetan_mastiff': 'others',
    'spider_monkey': 'others',
    'Doberman': 'others',
    'Boston_bull': 'others',
    'Greater_Swiss_Mountain_dog': 'others',
    'Appenzeller': 'others',
    'Shih-Tzu': 'others',
    'Irish_water_spaniel': 'others',
    'Pomeranian': 'others',
    'Bedlington_terrier': 'others',
    'warthog': 'others',
    'Arabian_camel': 'others',
    'siamang': 'others',
    'miniature_schnauzer': 'others',
    'collie': 'others',
    'golden_retriever': 'others',
    'Irish_terrier': 'others',
    'affenpinscher': 'others',
    'Border_collie': 'others',
    'hare': 'others',
    'boxer': 'others',
    'silky_terrier': 'others',
    'beagle': 'others',
    'Leonberg': 'others',
    'German_short-haired_pointer': 'others',
    'patas': 'others',
    'dhole': 'others',
    'baboon': 'others',
    'macaque': 'others',
    'Chesapeake_Bay_retriever': 'others',
    'bull_mastiff': 'others',
    'kuvasz': 'others',
    'capuchin': 'others',
    'pug': 'others',
    'curly-coated_retriever': 'others',
    'Norwich_terrier': 'others',
    'flat-coated_retriever': 'others',
    'hog': 'others',
    'keeshond': 'others',
    'Eskimo_dog': 'others',
    'Brittany_spaniel': 'others',
    'standard_poodle': 'others',
    'Lakeland_terrier': 'others',
    'snow_leopard': 'others',
    'Gordon_setter': 'others',
    'dingo': 'others',
    'standard_schnauzer': 'others',
    'hamster': 'others',
    'Tibetan_terrier': 'others',
    'Arctic_fox': 'others',
    'wire-haired_fox_terrier': 'others',
    'basset': 'others',
    'water_buffalo': 'others',
    'American_black_bear': 'others',
    'Angora': 'others',
    'bison': 'others',
    'howler_monkey': 'others',
    'hippopotamus': 'others',
    'chow': 'others',
    'giant_panda': 'others',
    'American_Staffordshire_terrier': 'others',
    'Shetland_sheepdog': 'others',
    'Great_Pyrenees': 'others',
    'Chihuahua': 'others',
    'tabby': 'others',
    'marmoset': 'others',
    'Labrador_retriever': 'others',
    'Saint_Bernard': 'others',
    'armadillo': 'others',
    'Samoyed': 'others',
    'bluetick': 'others',
    'redbone': 'others',
    'polecat': 'others',
    'marmot': 'others',
    'kelpie': 'others',
    'gibbon': 'others',
    'llama': 'others',
    'miniature_pinscher': 'others',
    'wood_rabbit': 'others',
    'Italian_greyhound': 'others',
    'lion': 'others',
    'cocker_spaniel': 'others',
    'Irish_setter': 'others',
    'dugong': 'others',
    'Indian_elephant': 'others',
    'beaver': 'others',
    'Sussex_spaniel': 'others',
    'Pembroke': 'others',
    'Blenheim_spaniel': 'others',
    'Madagascar_cat': 'others',
    'Rhodesian_ridgeback': 'others',
    'lynx': 'others',
    'African_hunting_dog': 'others',
    'langur': 'others',
    'Ibizan_hound': 'others',
    'timber_wolf': 'others',
    'cheetah': 'others',
    'English_foxhound': 'others',
    'briard': 'others',
    'sloth_bear': 'others',
    'Border_terrier': 'others',
    'German_shepherd': 'others',
    'otter': 'others',
    'koala': 'others',
    'tusker': 'others',
    'echidna': 'others',
    'wallaby': 'others',
    'platypus': 'others',
    'wombat': 'others',
    'revolver': 'others',
    'umbrella': 'others',
    'schooner': 'others',
    'soccer_ball': 'others',
    'accordion': 'others',
    'ant': 'others',
    'starfish': 'others',
    'chambered_nautilus': 'others',
    'grand_piano': 'others',
    'laptop': 'others',
    'strawberry': 'others',
    'airliner': 'others',
    'warplane': 'others',
    'airship': 'others',
    'balloon': 'others',
    'space_shuttle': 'others',
    'fireboat': 'others',
    'gondola': 'others',
    'speedboat': 'others',
    'lifeboat': 'others',
    'canoe': 'others',
    'yawl': 'others',
    'catamaran': 'others',
    'trimaran': 'others',
    'container_ship': 'others',
    'liner': 'others',
    'pirate': 'others',
    'aircraft_carrier': 'others',
    'submarine': 'others',
    'wreck': 'others',
    'half_track': 'others',
    'missile': 'others',
    'bobsled': 'others',
    'dogsled': 'others',
     'cradle': 'others',
    'crib': 'others',
    'four-poster': 'others',
    'bookcase': 'others',
    'china_cabinet': 'others',
    'medicine_chest': 'others',
    'chiffonier': 'others',
    'table_lamp': 'others',
    'file': 'others',
    'park_bench': 'others',
    'barber_chair': 'others',
    'throne': 'others',
    'folding_chair': 'others',
    'rocking_chair': 'others',
    'studio_couch': 'others',
    'toilet_seat': 'others',
    'desk': 'others',
    'pool_table': 'others',
    'dining_table': 'others',
    'entertainment_center': 'others',
    'wardrobe': 'others',
    'Granny_Smith': 'others',
    'orange': 'others',
    'lemon': 'others',
    'fig': 'others',
    'pineapple': 'others',
    'banana': 'others',
    'jackfruit': 'others',
    'custard_apple': 'others',
    'pomegranate': 'others',
    'acorn': 'others',
    'hip': 'others',
    'ear': 'others',
    'rapeseed': 'others',
    'corn': 'others',
    'buckeye': 'others',
    'organ': 'others',
    'upright': 'others',
    'chime': 'others',
    'drum': 'others',
    'gong': 'others',
    'maraca': 'others',
    'marimba': 'others',
    'steel_drum': 'others',
    'banjo': 'others',
    'cello': 'others',
    'violin': 'others',
    'harp': 'others',
    'acoustic_guitar': 'others',
    'electric_guitar': 'others',
    'cornet': 'others',
    'French_horn': 'others',
    'trombone': 'others',
    'harmonica': 'others',
    'ocarina': 'others',
    'panpipe': 'others',
    'bassoon': 'others',
    'oboe': 'others',
    'sax': 'others',
    'flute': 'others',
    'daisy': 'others',
    'yellow_lady\'s_slipper': 'others',
    'cliff': 'others',
    'valley': 'others',
    'alp': 'others',
    'volcano': 'others',
    'promontory': 'others',
    'sandbar': 'others',
    'coral_reef': 'others',
    'lakeside': 'others',
    'seashore': 'others',
    'geyser': 'others',
    'hatchet': 'others',
    'cleaver': 'others',
    'letter_opener': 'others',
    'plane': 'others',
    'power_drill': 'others',
    'lawn_mower': 'others',
    'hammer': 'others',
    'corkscrew': 'others',
    'can_opener': 'others',
    'plunger': 'others',
    'screwdriver': 'others',
    'shovel': 'others',
    'plow': 'others',
    'chain_saw': 'others',
    'cock': 'others',
    'hen': 'others',
    'ostrich': 'others',
    'brambling': 'others',
    'goldfinch': 'others',
    'house_finch': 'others',
    'junco': 'others',
    'indigo_bunting': 'others',
    'robin': 'others',
    'bulbul': 'others',
    'jay': 'others',
    'magpie': 'others',
    'chickadee': 'others',
    'water_ouzel': 'others',
    'kite': 'others',
    'bald_eagle': 'others',
    'vulture': 'others',
    'great_grey_owl': 'others',
    'black_grouse': 'others',
    'ptarmigan': 'others',
    'ruffed_grouse': 'others',
    'prairie_chicken': 'others',
    'peacock': 'others',
    'quail': 'others',
    'partridge': 'others',
    'African_grey': 'others',
    'macaw': 'others',
    'sulphur-crested_cockatoo': 'others',
    'lorikeet': 'others',
    'coucal': 'others',
    'bee_eater': 'others',
    'hornbill': 'others',
    'hummingbird': 'others',
    'jacamar': 'others',
    'toucan': 'others',
    'drake': 'others',
    'red-breasted_merganser': 'others',
    'goose': 'others',
    'black_swan': 'others',
    'white_stork': 'others',
    'black_stork': 'others',
    'spoonbill': 'others',
    'flamingo': 'others',
    'American_egret': 'others',
    'little_blue_heron': 'others',
    'bittern': 'others',
    'crane': 'others',
    'limpkin': 'others',
    'American_coot': 'others',
    'bustard': 'others',
    'ruddy_turnstone': 'others',
    'red-backed_sandpiper': 'others',
    'redshank': 'others',
    'dowitcher': 'others',
    'oystercatcher': 'others',
    'European_gallinule': 'others',
    'pelican': 'others',
    'king_penguin': 'others',
    'albatross': 'others',
    'great_white_shark': 'others',
    'tiger_shark': 'others',
    'hammerhead': 'others',
    'electric_ray': 'others',
    'stingray': 'others',
    'barracouta': 'others',
    'coho': 'others',
    'tench': 'others',
    'goldfish': 'others',
     'eel': 'others',
    'rock_beauty': 'others',
    'anemone_fish': 'others',
    'lionfish': 'others',
    'puffer': 'others',
    'sturgeon': 'others',
    'gar': 'others',
    'loggerhead': 'others',
    'leatherback_turtle': 'others',
    'mud_turtle': 'others',
    'terrapin': 'others',
    'box_turtle': 'others',
    'banded_gecko': 'others',
    'common_iguana': 'others',
    'American_chameleon': 'others',
    'whiptail': 'others',
    'agama': 'others',
    'frilled_lizard': 'others',
    'alligator_lizard': 'others',
    'Gila_monster': 'others',
    'green_lizard': 'others',
    'African_chameleon': 'others',
    'Komodo_dragon': 'others',
    'triceratops': 'others',
    'African_crocodile': 'others',
    'American_alligator': 'others',
    'thunder_snake': 'others',
    'ringneck_snake': 'others',
    'hognose_snake': 'others',
    'green_snake': 'others',
    'king_snake': 'others',
    'garter_snake': 'others',
    'water_snake': 'others',
    'vine_snake': 'others',
    'night_snake': 'others',
    'boa_constrictor': 'others',
    'rock_python': 'others',
    'Indian_cobra': 'others',
    'green_mamba': 'others',
    'sea_snake': 'others',
    'horned_viper': 'others',
    'diamondback': 'others',
    'sidewinder': 'others',
    'European_fire_salamander': 'others',
    'common_newt': 'others',
    'eft': 'others',
    'spotted_salamander': 'others',
    'axolotl': 'others',
    'bullfrog': 'others',
    'tree_frog': 'others',
    'tailed_frog': 'others',
    'whistle': 'others',
    'wing': 'others',
    'paintbrush': 'others',
    'hand_blower': 'others',
    'oxygen_mask': 'others',
    'snorkel': 'others',
    'loudspeaker': 'others',
    'microphone': 'others',
    'screen': 'others',
    'mouse': 'others',
    'electric_fan': 'others',
    'oil_filter': 'others',
    'strainer': 'others',
    'space_heater': 'others',
    'stove': 'others',
    'guillotine': 'others',
    'barometer': 'others',
    'rule': 'others',
    'odometer': 'others',
    'scale': 'others',
    'analog_clock': 'others',
    'digital_clock': 'others',
    'wall_clock': 'others',
    'hourglass': 'others',
    'sundial': 'others',
    'parking_meter': 'others',
    'stopwatch': 'others',
    'digital_watch': 'others',
    'stethoscope': 'others',
    'syringe': 'others',
    'magnetic_compass': 'others',
    'binoculars': 'others',
    'projector': 'others',
    'sunglasses': 'others',
    'loupe': 'others',
    'radio_telescope': 'others',
    'bow': 'others',
    'cannon': 'others',
    'assault_rifle': 'others',
    'rifle': 'others',
    'projectile': 'others',
    'computer_keyboard': 'others',
    'typewriter_keyboard': 'others',
    'crane': 'others',
    'lighter': 'others',
    'abacus': 'others',
    'cash_machine': 'others',
    'slide_rule': 'others',
    'desktop_computer': 'others',
    'hand-held_computer': 'others',
    'notebook': 'others',
    'web_site': 'others',
    'harvester': 'others',
    'thresher': 'others',
    'printer': 'others',
    'slot': 'others',
    'vending_machine': 'others',
    'sewing_machine': 'others',
    'joystick': 'others',
    'switch': 'others',
    'hook': 'others',
    'car_wheel': 'others',
    'paddlewheel': 'others',
    'pinwheel': 'others',
    'potter\'s_wheel': 'others',
    'gas_pump': 'others',
    'carousel': 'others',
    'swing': 'others',
    'reel': 'others',
    'radiator': 'others',
    'puck': 'others',
    'hard_disc': 'others',
    'sunglass': 'others',
    'pick': 'others',
    'car_mirror': 'others',
    'solar_dish': 'others',
    'remote_control': 'others',
    'disk_brake': 'others',
    'buckle': 'others',
    'hair_slide': 'others',
    'knot': 'others',
    'combination_lock': 'others',
    'padlock': 'others',
    'nail': 'others',
    'safety_pin': 'others',
    'screw': 'others',
    'muzzle': 'others',
    'seat_belt': 'others',
    'ski': 'others',
    'candle': 'others',
    'spotlight': 'others',
    'torch': 'others',
    'neck_brace': 'others',
    'pier': 'others',
    'tripod': 'others',
     'maypole': 'others',
    'mousetrap': 'others',
    'spider_web': 'others',
    'trilobite': 'others',
    'harvestman': 'others',
    'scorpion': 'others',
    'black_and_gold_garden_spider': 'others',
    'barn_spider': 'others',
    'garden_spider': 'others',
    'black_widow': 'others',
    'tarantula': 'others',
    'wolf_spider': 'others',
    'tick': 'others',
    'centipede': 'others',
    'isopod': 'others',
    'Dungeness_crab': 'others',
    'rock_crab': 'others',
    'fiddler_crab': 'others',
    'king_crab': 'others',
    'American_lobster': 'others',
    'spiny_lobster': 'others',
    'crayfish': 'others',
    'hermit_crab': 'others',
    'tiger_beetle': 'others',
    'ladybug': 'others',
    'ground_beetle': 'others',
    'long-horned_beetle': 'others',
    'leaf_beetle': 'others',
    'dung_beetle': 'others',
    'rhinoceros_beetle': 'others',
    'weevil': 'others',
    'fly': 'others',
    'bee': 'others',
    'grasshopper': 'others',
    'cricket': 'others',
    'walking_stick': 'others',
    'cockroach': 'others',
    'mantis': 'others',
    'cicada': 'others',
    'leafhopper': 'others',
    'lacewing': 'others',
    'dragonfly': 'others',
    'damselfly': 'others',
    'admiral': 'others',
    'ringlet': 'others',
    'monarch': 'others',
    'cabbage_butterfly': 'others',
    'sulphur_butterfly': 'others',
    'lycaenid': 'others',
    'jellyfish': 'others',
    'sea_anemone': 'others',
    'brain_coral': 'others',
    'flatworm': 'others',
    'nematode': 'others',
    'conch': 'others',
    'snail': 'others',
    'slug': 'others',
    'sea_slug': 'others',
    'chiton': 'others',
    'sea_urchin': 'others',
    'sea_cucumber': 'others',
    'iron': 'others',
    'espresso_maker': 'others',
    'microwave': 'others',
    'Dutch_oven': 'others',
    'rotisserie': 'others',
    'toaster': 'others',
    'waffle_iron': 'others',
    'vacuum': 'others',
    'dishwasher': 'others',
    'refrigerator': 'others',
    'washer': 'others',
    'Crock_Pot': 'others',
    'frying_pan': 'others',
    'wok': 'others',
    'caldron': 'others',
    'coffeepot': 'others',
    'teapot': 'others',
    'spatula': 'others',
    'altar': 'others',
    'triumphal_arch': 'others',
    'patio': 'others',
    'steel_arch_bridge': 'others',
    'suspension_bridge': 'others',
    'viaduct': 'others',
    'barn': 'others',
    'greenhouse': 'others',
    'palace': 'others',
    'monastery': 'others',
    'library': 'others',
    'apiary': 'others',
    'boathouse': 'others',
    'church': 'others',
    'mosque': 'others',
    'stupa': 'others',
    'planetarium': 'others',
    'restaurant': 'others',
    'cinema': 'others',
    'home_theater': 'others',
    'lumbermill': 'others',
    'coil': 'others',
    'obelisk': 'others',
    'totem_pole': 'others',
    'castle': 'others',
    'prison': 'others',
    'grocery_store': 'others',
    'bakery': 'others',
    'barbershop': 'others',
    'bookshop': 'others',
    'butcher_shop': 'others',
    'confectionery': 'others',
    'shoe_shop': 'others',
    'tobacco_shop': 'others',
    'toyshop': 'others',
    'fountain': 'others',
    'cliff_dwelling': 'others',
    'yurt': 'others',
    'dock': 'others',
    'brass': 'others',
    'megalith': 'others',
    'bannister': 'others',
    'breakwater': 'others',
    'dam': 'others',
     'chainlink_fence': 'others',
    'picket_fence': 'others',
    'worm_fence': 'others',
    'stone_wall': 'others',
    'grille': 'others',
    'sliding_door': 'others',
    'turnstile': 'others',
    'mountain_tent': 'others',
    'scoreboard': 'others',
    'honeycomb': 'others',
    'plate_rack': 'others',
    'pedestal': 'others',
    'beacon': 'others',
    'mashed_potato': 'others',
    'ping-pong_ball': 'others',
    'pencil_box': 'others',
    'pay-phone': 'others',
    'consomme': 'others',
    'apron': 'others',
    'punching_bag': 'others',
    'backpack': 'others',
    'groom': 'others',
    'bearskin': 'others',
    'pencil_sharpener': 'others',
    'broom': 'others',
    'mosquito_net': 'others',
    'abaya': 'others',
    'mortarboard': 'others',
    'poncho': 'others',
    'crutch': 'others',
    'Polaroid_camera': 'others',
    'space_bar': 'others',
    'cup': 'others',
    'racket': 'others',
    'traffic_light': 'others',
    'quill': 'others',
    'radio': 'others',
    'dough': 'others',
    'cuirass': 'others',
    'military_uniform': 'others',
    'lipstick': 'others',
    'shower_cap': 'others',
    'monitor': 'others',
    'oscilloscope': 'others',
    'mitten': 'others',
    'brassiere': 'others',
    'French_loaf': 'others',
    'vase': 'others',
    'milk_can': 'others',
    'rugby_ball': 'others',
    'paper_towel': 'others',
    'earthstar': 'others',
    'envelope': 'others',
    'miniskirt': 'others',
    'cowboy_hat': 'others',
    'trolleybus': 'others',
    'perfume': 'others',
    'bathtub': 'others',
    'hotdog': 'others',
    'coral_fungus': 'others',
    'bullet_train': 'others',
    'pillow': 'others',
    'toilet_tissue': 'others',
    'cassette': 'others',
    "carpenter's_kit": 'others',
    'ladle': 'others',
    'stinkhorn': 'others',
    'lotion': 'others',
    'hair_spray': 'others',
    'academic_gown': 'others',
    'dome': 'others',
    'crate': 'others',
    'wig': 'others',
    'burrito': 'others',
    'pill_bottle': 'others',
    'chain_mail': 'others',
    'theater_curtain': 'others',
    'window_shade': 'others',
    'barrel': 'others',
    'washbasin': 'others',
    'ballpoint': 'others',
    'basketball': 'others',
    'bath_towel': 'others',
    'cowboy_boot': 'others',
    'gown': 'others',
    'window_screen': 'others',
    'agaric': 'others',
    'cellular_telephone': 'others',
    'nipple': 'others',
    'barbell': 'others',
    'mailbox': 'others',
    'lab_coat': 'others',
    'fire_screen': 'others',
    'ping-pong_ball': 'others',
    'pencil_box': 'others',
    'pay-phone': 'others',
    'consomme': 'others',
    'apron': 'others',
    'punching_bag': 'others',
    'backpack': 'others',
    'groom': 'others',
    'bearskin': 'others',
    'pencil_sharpener': 'others',
    'broom': 'others',
    'mosquito_net': 'others',
    'abaya': 'others',
    'mortarboard': 'others',
    'poncho': 'others',
    'crutch': 'others',
    'Polaroid_camera': 'others',
    'space_bar': 'others',
    'cup': 'others',
    'racket': 'others',
    'traffic_light': 'others',
    'quill': 'others',
    'radio': 'others',
    'dough': 'others',
    'cuirass': 'others',
    'military_uniform': 'others',
    'lipstick': 'others',
    'shower_cap': 'others',
    'monitor': 'others',
    'oscilloscope': 'others',
    'mitten': 'others',
    'brassiere': 'others',
    'French_loaf': 'others',
    'vase': 'others',
    'milk_can': 'others',
    'rugby_ball': 'others',
    'paper_towel': 'others',
    'earthstar': 'others',
    'envelope': 'others',
    'miniskirt': 'others',
    'cowboy_hat': 'others',
    'trolleybus': 'others',
    'perfume': 'others',
    'bathtub': 'others',
    'hotdog': 'others',
    'coral_fungus': 'others',
    'bullet_train': 'others',
    'pillow': 'others',
    'toilet_tissue': 'others',
    'cassette': 'others',
    "carpenter's_kit": 'others',
    'ladle': 'others',
    'stinkhorn': 'others',
    'lotion': 'others',
    'hair_spray': 'others',
    'academic_gown': 'others',
    'dome': 'others',
    'crate': 'others',
    'wig': 'others',
    'burrito': 'others',
    'pill_bottle': 'others',
    'chain_mail': 'others',
    'theater_curtain': 'others',
    'window_shade': 'others',
    'barrel': 'others',
    'washbasin': 'others',
    'ballpoint': 'others',
    'basketball': 'others',
    'bath_towel': 'others',
    'cowboy_boot': 'others',
    'gown': 'others',
    'window_screen': 'others',
    'agaric': 'others',
    'cellular_telephone': 'others',
    'nipple': 'others',
    'barbell': 'others',
    'mailbox': 'others',
    'lab_coat': 'others',
    'fire_screen': 'others',
    'minibus': 'others',
    'packet': 'others',
    'maze': 'others',
    'pole': 'others',
    'horizontal_bar': 'others',
    'sombrero': 'others',
    'pickelhaube': 'others',
    'rain_barrel': 'others',
    'wallet': 'others',
    'cassette_player': 'others',
    'comic_book': 'others',
    'piggy_bank': 'others',
    'street_sign': 'others',
    'bell_cote': 'others',
    'fountain_pen': 'others',
    'Windsor_tie': 'others',
    'volleyball': 'others',
    'overskirt': 'others',
    'sarong': 'others',
    'purse': 'others',
    'bolo_tie': 'others',
    'bib': 'others',
     'parachute': 'others',
    'sleeping_bag': 'others',
    'television': 'others',
    'swimming_trunks': 'others',
    'measuring_cup': 'others',
    'espresso': 'others',
    'pizza': 'others',
    'breastplate': 'others',
    'shopping_basket': 'others',
    'wooden_spoon': 'others',
    'saltshaker': 'others',
    'chocolate_sauce': 'others',
    'ballplayer': 'others',
    'goblet': 'others',
    'gyromitra': 'others',
    'stretcher': 'others',
    'water_bottle': 'others',
    'dial_telephone': 'others',
    'soap_dispenser': 'others',
    'jersey': 'others',
    'school_bus': 'others',
    'jigsaw_puzzle': 'others',
    'plastic_bag': 'others',
    'reflex_camera': 'others',
    'diaper': 'others',
    'Band_Aid': 'others',
    'ice_lolly': 'others',
    'velvet': 'others',
    'tennis_ball': 'others',
    'gasmask': 'others',
    'doormat': 'others',
    'Loafer': 'others',
    'ice_cream': 'others',
    'pretzel': 'others',
    'quilt': 'others',
    'maillot': 'others',
    'tape_player': 'others',
    'clog': 'others',
    'iPod': 'others',
    'bolete': 'others',
    'scuba_diver': 'others',
    'pitcher': 'others',
    'matchstick': 'others',
    'bikini': 'others',
    'sock': 'others',
    'CD_player': 'others',
    'lens_cap': 'others',
    'thatch': 'others',
    'vault': 'others',
    'beaker': 'others',
    'bubble': 'others',
    'cheeseburger': 'others',
    'parallel_bars': 'others',
    'flagpole': 'others',
    'coffee_mug': 'others',
    'rubber_eraser': 'others',
    'stole': 'others',
    'carbonara': 'others',
    'dumbbell': 'others'
}


**Ufff!!! It is so time consuming**
## Now let me match custom_mapping with imagenet_mapping to get the desired output

In [51]:
print("Top Prediction:")
imagenet_id, label, score = decoded_predictions[0]
print(f"{imagenet_id} {label} ({score:.2f})")

Top Prediction:
n03791053 motor_scooter (0.23)


In [52]:
def map_imagenet_label_to_class(predicted_label, imagenet_mapping):
    for imagenet_id, mapped_class in imagenet_mapping.items():
        if predicted_label.startswith(imagenet_id):
            return mapped_class
    return 'unknown'

mapped_class = map_imagenet_label_to_class(label, imagenet_mapping)
print(f'Mapped Class: {mapped_class}')


Mapped Class: bike


# **Ok!!!**

This is what I believe the assignment is about.

Due to time constraints I am ending it here because I have a model to deploy

## In future if need arises or if this is the right way to approach the project we will continue further